In [5]:
import os
import rasterio
from rasterio.transform import from_bounds
from shapely.geometry import box, mapping
import geojson





In [6]:

# Directory containing all the 1km x 1km raster tiles
raster_directory = "/Volumes/Expansion/DTM_UNZIP/Belleville-DTM-A"

# List to store GeoJSON features
features = []

# Initialize an empty CRS variable
crs = None

# Iterate over each file in the directory
for filename in os.listdir(raster_directory):
    if filename.endswith('.tif'):  # Check if the file is a raster
        raster_path = os.path.join(raster_directory, filename)
        
        with rasterio.open(raster_path) as src:
            # Get bounds of the raster
            bounds = src.bounds
            
            # Create a polygon (square) from the bounds
            polygon = box(bounds.left, bounds.bottom, bounds.right, bounds.top)
            
            # Create a GeoJSON feature for this tile
            feature = geojson.Feature(
                geometry=mapping(polygon),
                properties={"filename": filename}
            )
            
            features.append(feature)
            
            # Store the CRS from the first raster file
            if crs is None:
                crs = src.crs

# Create a GeoJSON FeatureCollection
feature_collection = geojson.FeatureCollection(features)

# Check if CRS is found and add it to the FeatureCollection
if crs:
    feature_collection['crs'] = {
        "type": "name",
        "properties": {
            "name": crs.to_string()  # Convert CRS to a string format for GeoJSON
        }
    }




GeoJSON file saved as output_tiles.geojson


ERROR 1: PROJ: internal_proj_identify: /Users/b/miniforge3/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 3 is expected. It comes from another PROJ installation.


In [9]:
# Output GeoJSON file path
output_geojson = "output_tiles.geojson"

# Write the FeatureCollection to a GeoJSON file
with open(output_geojson, 'w') as f:
    geojson.dump(feature_collection, f, indent=2)

print(f"GeoJSON file saved as {output_geojson}")

GeoJSON file saved as output_tiles.geojson
